# FOMC Meeting Minutes

In this notebook, I have implemented a scraping code to retrieve the minutes of the Federal Open Market Committee (FOMC) meetings from the official Federal Reserve website. To accomplish this, I have compiled a comprehensive list of all the days on which these meetings occurred. It is worth noting that, in most cases, the meetings were held over two days, and I have manually curated the dates based on the information found in the minutes' URL.

In the process, I rely on two custom-made functions: `paragraph_processor` and `text_processor`. These functions are crucial in processing the HTML files and performing essential preprocessing steps on the data. For detailed insights into the functionalities and usage of these functions, please refer to the documentation provided within the `functions.py` file.

In [1]:
import pandas as pd
import spacy
import pickle
import time

In [2]:
from functions import text_processor, paragraph_processor

In [3]:
minutes_dates = [
    '19930203', '19930323', '19930518', '19930707', '19930817', '19930921', '19931116', '19931221', # 1993 FOMC minutes   
    '19940204', '19940322', '19940517', '19940706', '19940816', '19940927', '19941115', '19941220', # 1994 FOMC minutes 
    '19950201', '19950328', '19950523', '19950706', '19950822', '19950926', '19951115', '19951219', # 1995 FOMC minutes  
    '19960130', '19960326', '19960521', '19960702', '19960820', '19960924', '19961113', '19961217', # 1996 FOMC minutes    
    '19970204', '19970325', '19970520', '19970701', '19970819', '19970930', '19971112', '19971216', # 1997 FOMC minutes    
    '19980203', '19980331', '19980519', '19980630', '19980818', '19980929', '19981117', '19981222', # 1998 FOMC minutes    
    '19990202', '19990330', '19990518', '19990629', '19990824', '19991005', '19991116', '19991221', # 1999 FOMC minutes    
    '20000202', '20000321', '20000516', '20000628', '20000822', '20001003', '20001115', '20001219', # 2000 FOMC minutes 
    '20010131', '20010320', '20010515', '20010627', '20010821', '20011002', '20011106', '20011211', # 2001 FOMC minutes 
    '20020130', '20020319', '20020507', '20020626', '20020813', '20020924', '20021106', '20021210', # 2002 FOMC minutes 
    '20030129', '20030318', '20030506', '20030625', '20030812', '20030916', '20031028', '20031209', # 2003 FOMC minutes  
    '20040128', '20040316', '20040504', '20040630', '20040810', '20040921', '20041110', '20041214', # 2004 FOMC minutes 
    '20050202', '20050322', '20050503', '20050630', '20050809', '20050920', '20051101', '20051213', # 2005 FOMC minutes 
    '20060131', '20060328', '20060510', '20060629', '20060808', '20060920', '20061025', '20061212', # 2006 FOMC minutes   
    '20070131', '20070321', '20070509', '20070628', '20070807', '20070918', '20071031', '20071211', # 2007 FOMC minutes 
    '20080130', '20080318', '20080430', '20080625', '20080805', '20080916', '20081029', '20081216', # 2008 FOMC minutes     
    '20090128', '20090318', '20090429', '20090624', '20090812', '20090923', '20091104', '20091216', # 2009 FOMC minutes 
    '20100127', '20100316', '20100428', '20100623', '20100810', '20100921', '20101103', '20101214', # 2010 FOMC minutes 
    '20110126', '20110315', '20110427', '20110622', '20110809', '20110921', '20111102', '20111213', # 2011 FOMC minutes 
    '20120125', '20120313', '20120425', '20120620', '20120801', '20120913', '20121024', '20121212', # 2012 FOMC minutes 
    '20130130', '20130320', '20130501', '20130619', '20130731', '20130918', '20131030', '20131218', # 2013 FOMC minutes 
    '20140129', '20140319', '20140430', '20140618', '20140730', '20140917', '20141029', '20141217', # 2014 FOMC minutes                   
    '20150128', '20150318', '20150429', '20150617', '20150729', '20150917', '20151028', '20151216', # 2015 FOMC minutes    
    '20160127', '20160316', '20160427', '20160615', '20160727', '20160921', '20161102', '20161214', # 2016 FOMC minutes
    '20170201', '20170315', '20170503', '20170614', '20170726', '20170920', '20171101', '20171213', # 2017 FOMC minutes
    '20180131', '20180321', '20180502', '20180613', '20180801', '20180926', '20181108', '20181219', # 2018 FOMC minutes
    '20190130', '20190320', '20190501', '20190619', '20190731', '20190918', '20191030', '20191211', # 2019 FOMC minutes
    '20200129', '20200315', '20200429', '20200610', '20200729', '20200916', '20201105', '20201216', # 2020 FOMC minutes
    '20210127', '20210317', '20210428', '20210616', '20210728', '20210922', '20211103', '20211215', # 2021 FOMC minutes
    '20220126', '20220316', '20220504', '20220615', '20220727', '20220921', '20221102', '20221214', # 2022 FOMC minutes
    '20230201', '20230322', '20230503', '20230614' # 2023 FOMC minutes
]


In [4]:
with open('../data/minutes_dates.pkl', 'wb') as file:
    pickle.dump(minutes_dates, file)

In [5]:
# Load the "en_core_web_lg" model from spaCy for natural language processing tasks
nlp = spacy.load("en_core_web_lg")

# Modify the maximum allowable length of text that the NLP pipeline can process to 1500000
# This adjustment ensures the pipeline can handle longer texts (in characters) during NLP tasks
# given that fomc minutes tend to be very long. 
nlp.max_length = 1500000

In [9]:
start_time = time.time()

# Create a dictionary called 'fomcminutes' using a dictionary comprehension.
# The dictionary will store processed text data from FOMC meeting minutes, with dates as keys and processed text as values.
# 'minutes_dates' is a list containing all the dates when FOMC meetings took place.

fomcminutes = {date: text_processor(date) for date in minutes_dates}
# For each date in the 'minutes_dates' list, the 'text_processor' function is called to process the meeting minutes text.
# The output of the 'text_processor' function (processed text data) is then stored in the 'fomcminutes' dictionary
# with the corresponding date as the key.
# The resulting 'fomcminutes' dictionary organizes the processed text data based on the date of each FOMC meeting.

end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

Execution time: 383.9797468185425 seconds


In [10]:
# Create a DataFrame 'fomc' from the 'fomcminutes' dictionary.
# The DataFrame will have dates as rows and columns as features, containing the processed text data from FOMC meeting minutes.
fomc = pd.DataFrame(fomcminutes).transpose()

# Convert the index of the DataFrame to datetime format.
fomc.index = pd.to_datetime(fomc.index)

# Rename the columns of the DataFrame to have descriptive feature names.
fomc.columns = ['minutes_paragraphs', 'paragraphs_length', 'minutes_text', 'text_length']

fomc.head()

,minutes_paragraphs,paragraphs_length,minutes_text,text_length
1993-02-03,"[[meeting, federal, open, market, committee, h...","[12, 15, 24, 29, 12, 32, 37, 32, 14, 16, 82, 5...",meeting federal open market committee hold off...,4437
1993-03-23,"[[meeting, federal, open, market, committee, h...","[11, 13, 64, 23, 24, 28, 60, 51, 64, 56, 100, ...",meeting federal open market committee hold off...,2789
1993-05-18,"[[meeting, federal, open, market, committee, h...","[11, 26, 19, 25, 27, 62, 46, 54, 37, 89, 56, 6...",meeting federal open market committee hold off...,2354
1993-07-07,"[[meeting, federal, open, market, committee, h...","[12, 23, 20, 24, 31, 46, 39, 47, 60, 63, 60, 4...",meeting federal open market committee hold off...,3181
1993-08-17,"[[meeting, federal, open, market, committee, h...","[11, 14, 17, 21, 24, 28, 45, 43, 50, 51, 63, 6...",meeting federal open market committee hold off...,2225


In [11]:
fomc.to_pickle('../data/fomc_data.pkl')